In [ ]:
# Fix locale issue
import locale
import os
os.environ['LANG'] = 'en_US.UTF-8'
os.environ['LC_ALL'] = 'en_US.UTF-8'
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Install required packages
!apt-get update
!apt-get install -y locales
!locale-gen en_US.UTF-8
!pip install ultralytics roboflow
!nvidia-smi # Check GPU availability

import os
from ultralytics import YOLO
from roboflow import Roboflow
import matplotlib.pyplot as plt

rf = Roboflow(api_key="Hv8TM41Le0zo9uA4SIPC")
project = rf.workspace("yanelys").project("clothing-segmentation-dataset")
version = project.version(9)
dataset = version.download("yolov8")

DATASET_YAML_PATH = "/content/Clothing-Segmentation-DataSet-9/data.yaml"

model = YOLO('yolov8l-seg.pt')

results = model.train(
    data=DATASET_YAML_PATH,
    epochs=150,
    imgsz=640,
    batch=14,
    patience=10,
    project='clothing_segmentation',
    name='optimized_training',
    cos_lr=True,
    lr0=0.0003,
    lrf=0.0001,
    warmup_epochs=5,
    momentum=0.937,
    weight_decay=0.0004,
    augment=True,
    mixup=0.15,
    mosaic=0.8,
    degrees=15.0,
    translate=0.2,
    scale=0.6,
    shear=2.0,
    perspective=0.0015,
    flipud=0.5,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    cache=False,
    workers=8,
    device=0
)

try:
    metrics = results.results
    plt.figure(figsize=(15, 10))

    plt.subplot(2, 2, 1)
    if 'train/box_loss' in metrics:
        plt.plot(metrics['train/box_loss'], label='box_loss')
    if 'train/seg_loss' in metrics:
        plt.plot(metrics['train/seg_loss'], label='seg_loss')
    if 'train/cls_loss' in metrics:
        plt.plot(metrics['train/cls_loss'], label='cls_loss')
    if 'train/dfl_loss' in metrics:
        plt.plot(metrics['train/dfl_loss'], label='dfl_loss')
    plt.title('Training Losses')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(2, 2, 2)
    if 'metrics/mAP50(B)' in metrics:
        plt.plot(metrics['metrics/mAP50(B)'], label='mAP50')
    if 'metrics/mAP50-95(B)' in metrics:
        plt.plot(metrics['metrics/mAP50-95(B)'], label='mAP50-95')
    plt.title('mAP Metrics')
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    plt.legend()
    plt.grid(True)

    plt.subplot(2, 2, 3)
    if 'metrics/precision(B)' in metrics:
        plt.plot(metrics['metrics/precision(B)'], label='Precision')
    if 'metrics/recall(B)' in metrics:
        plt.plot(metrics['metrics/recall(B)'], label='Recall')
    plt.title('Precision and Recall')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"Error in plotting: {e}")
    if 'metrics' in locals():
        print("\nAvailable metrics keys:", metrics.keys())

print("\nValidation Results:")
val_results = model.val(data=DATASET_YAML_PATH, split='test')
print("\nDetailed Metrics:")
print(f"mAP50: {val_results.box.map50:.4f}")
print(f"mAP50-95: {val_results.box.map:.4f}")
print(f"Precision: {val_results.box.mp:.4f}")
print(f"Recall: {val_results.box.mr:.4f}")

model.export(format='onnx')
print("\nModel exported successfully!")

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,192 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,225 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/u


Extracting Dataset Version Zip to Clothing-Segmentation-DataSet-9 in yolov8:: 100%|██████████| 2180/2180 [00:00<00:00, 3019.28it/s]


100%|██████████| 88.1M/88.1M [00:02<00:00, 42.8MB/s]


Ultralytics 8.3.43 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8l-seg.pt, data=/content/Clothing-Segmentation-DataSet-9/data.yaml, epochs=150, time=None, patience=10, batch=14, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=clothing_segmentation, name=optimized_training, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cr

100%|██████████| 755k/755k [00:00<00:00, 150MB/s]


...

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  88%|████████▊ | 7/8 [00:07<00:00,  1.08it/s]

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]


                   all        109        191      0.654      0.698      0.716      0.634      0.658      0.704      0.719      0.603
                  coat         17         20      0.593        0.5      0.564      0.513      0.593        0.5      0.564      0.423
                 dress         25         25      0.642      0.789      0.766      0.595      0.609      0.749      0.711      0.626
                jacket         16         16      0.784       0.68      0.757      0.696      0.784       0.68      0.757      0.548
                 pants         24         24      0.774      0.856      0.881      0.827      0.774      0.856      0.881        0.8
                 shirt         28         29      0.509      0.608      0.526      0.467      0.509      0.608      0.527      0.448
                shorts         22         22      0.859      0.909      0.935      0.826      0.902      0.955      0.985      0.795
                 skirt         27         27       0.53      0.741   

In [7]:
from google.colab import files

# Download the trained models
files.download('/content/clothing_segmentation/optimized_training/weights/best.pt')
# files.download('/content/clothing_segmentation/optimized_training/weights/best.onnx')

# # Create a simple plot of the final metrics
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))
# plt.bar(['mAP50', 'mAP50-95', 'Precision', 'Recall'],
#         [val_results.box.map50, val_results.box.map, val_results.box.mp, val_results.box.mr])
# plt.title('Final Model Metrics')
# plt.ylabel('Score')
# plt.ylim(0, 1.0)
# plt.savefig('final_metrics.png')
# files.download('final_metrics.png')

# print("Files should now be downloading in your browser.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>